In [20]:
#all imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
import pandas as pd
from tqdm import tqdm
import math

In [21]:
nodes = pd.read_csv("youtube_raw.csv").drop("id.1",axis = 1)

#eliminate recommendations
nodes = nodes.iloc[:,0:9]

In [22]:
#transform every column in list
for i in nodes.columns:
    globals()[i]=nodes[i].values.tolist()
    

In [23]:
#print all variables that are now available as lists
nodes.columns

Index(['id', 'uploader', 'age', 'category', 'length', 'views', 'rate',
       'ratings', 'comments'],
      dtype='object')

In [24]:
#read train set and train graph
train_edges = pd.read_csv("train_set.csv", index_col =0).values.tolist()
G_train = nx.read_edgelist("train.edgelist", delimiter=',')

#read test set and test graph
test_edges = pd.read_csv("test_set.csv", index_col =0).values.tolist()
G_test = nx.read_edgelist("test.edgelist", delimiter=',')

In [18]:
!pip install node2vec

  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1


In [ ]:
from node2vec import Node2Vec
# Precompute probabilities and generate walks
node2vec = Node2Vec(G_train, dimensions=64, walk_length=10, num_walks=20, workers=4)
# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)

Computing transition probabilities:   0%|          | 0/154677 [00:00<?, ?it/s]

In [ ]:
t10cu_emb = { n: list(map(float,model.wv.get_vector(n))) for n in G_train.nodes()}

In [10]:
# # WUvCeCflehk
# t10cu_emb.get('WUvCeCflehk')

[0.6851642727851868,
 -1.1165839433670044,
 -0.45379969477653503,
 -0.004886134527623653,
 0.03574543073773384,
 -0.5158502459526062,
 0.2497645616531372,
 -0.7186118364334106,
 -0.9843817353248596,
 0.15378302335739136,
 1.2988859415054321,
 -0.08178991079330444,
 -0.024460218846797943,
 -0.47229963541030884,
 -0.11182890087366104,
 0.21071544289588928,
 0.04947487637400627,
 0.6645092368125916,
 0.14232604205608368,
 0.06691809743642807,
 -0.23847316205501556,
 0.03428015857934952,
 -0.43026605248451233,
 -0.5684924125671387,
 -0.22894608974456787,
 -0.5256617069244385,
 -0.33328118920326233,
 -0.05468614399433136,
 -0.889137864112854,
 0.01730462536215782,
 0.09561730176210403,
 0.1865576058626175,
 -0.43307599425315857,
 0.0023650259245187044,
 -0.21049264073371887,
 -0.004525403026491404,
 0.14676356315612793,
 -0.9984727501869202,
 -0.4060112535953522,
 -0.010012958198785782,
 0.6407720446586609,
 0.1595916450023651,
 0.1647300273180008,
 -0.790908694267273,
 0.14037375152111053,

In [11]:
t10cu_emb.get('xuOtyEjQ-v4')

[0.9730386734008789,
 -1.207642912864685,
 -0.4464534521102905,
 -0.04092391952872276,
 0.19762003421783447,
 -0.4454960525035858,
 0.4121575355529785,
 -0.8269778490066528,
 -1.2874515056610107,
 -0.005511773284524679,
 1.513331651687622,
 -0.24635641276836395,
 -0.11321515589952469,
 -0.7641708850860596,
 -0.07965905219316483,
 0.15205566585063934,
 -0.0010589392622932792,
 0.5870465636253357,
 0.1347959339618683,
 0.13706085085868835,
 -0.4283437430858612,
 -0.0281562190502882,
 -0.40848591923713684,
 -0.5820826292037964,
 -0.30136609077453613,
 -0.7581773400306702,
 -0.41187137365341187,
 -0.10303853452205658,
 -1.001473307609558,
 0.08912406861782074,
 0.22901929914951324,
 0.13475264608860016,
 -0.4309629201889038,
 0.065004363656044,
 -0.07712556421756744,
 -0.1449437290430069,
 0.1585787683725357,
 -1.2553962469100952,
 -0.38721200823783875,
 0.009767760522663593,
 0.7659207582473755,
 0.05086977407336235,
 0.22026868164539337,
 -0.9354638457298279,
 0.26120132207870483,
 -1.17

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2): 
    feature_vec_1= np.array(feature_vec_1)
    feature_vec_2= np.array(feature_vec_2)
    return cosine_similarity((np.reshape(feature_vec_1 ,(1, -1))), np.reshape(feature_vec_2 ,(1, -1)))[0][0]

In [13]:
def similarity_features(d, source, target):
    feature_vec_1= d.get(source)
#     print(feature_vec_1)
    feature_vec_2= d.get(target)
#     print("2")
#     print(feature_vec_2)
    sim= get_cosine_similarity(feature_vec_1, feature_vec_2)
    return sim

In [14]:
# for i in tqdm(train_edges):
#     similarity_features(t10cu_emb, i[0],i[1])

x_train = []
y_train = []
for i in tqdm(train_edges):
    x_train.append(similarity_features(t10cu_emb, i[0],i[1]))
    y_train.append(int(i[2]))

100%|████████████████████████████████████████████████████████████████████████| 307274/307274 [00:49<00:00, 6240.02it/s]


In [ ]:
#train_Edges: ['xuOtyEjQ-v4', 'WUvCeCflehk', 1]
#for i in train edges, we need to compute similarity between the feature vectors of i[0] and i[1]
#then store these similarities in a list and append it to the features. 

In [8]:
# def katz_centrality(G):
#     phi = (1 + math.sqrt(5)) / 2.0  # largest eigenvalue of adj matrix
#     centrality = nx.katz_centrality(G_train, 1 / phi - 0.01)
#     for n, c in sorted(centrality.items()):
#         print(f"{n} {c:.2f}")
#     return centrality

In [15]:
x_train = pd.DataFrame(x_train, columns = ['similarities'])
            
x_train['y'] = y_train

In [16]:
# x_train.to_csv('similarities_train.csv', index=False)

In [17]:
#compute features for training set
x_test = []
y_test = []
for i in tqdm(test_edges):
    x_test.append(similarity_features(t10cu_emb, i[0],i[1]))
    y_test.append(int(i[2]))

100%|████████████████████████████████████████████████████████████████████████| 170706/170706 [00:29<00:00, 5790.84it/s]


In [18]:
x_test = pd.DataFrame(x_test, columns = ['similarities'])
            
x_test['y'] = y_test

In [19]:
x_test.to_csv('similarities_test.csv', index=False)